In [121]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

pts1 = gpd.read_file('../targets_0_sar/ITA/sar_ITA_e_v2.geojson')
pts2 = gpd.read_file('../targets_0_sar/ITA/sar_ITA_w_v2.geojson')
pts = pd.concat([pts1, pts2])
print(len(pts1))
print(len(pts2))
print(len(pts))

1435
1644
3079


In [122]:
#load images
n = 'italy'
folder = '/Users/Zack/0_thesis_bing/imagery_' + n + '/'
files = os.listdir(folder)
print(len(files))

#extract coordinates
x_list, y_list = [], []
for i in range(len(files)):
    #extract bottom right coordinate from image name
    img = files[i]
    x1 = float(img.split(',')[1])
    y1 = float(img.split(',')[2])
    x2 = float(img.split(',')[3])
    y2 = float((img.split(',')[4]).split(')')[0])
    x = (x1+x2)/2
    y = (y1+y2)/2
    x_list.append(x)
    y_list.append(y)

1289


In [123]:
geometry = [Point(i) for i in zip(y_list, x_list)]
missing = gpd.GeoDataFrame(geometry=geometry, crs={'init' :'EPSG:4326'})
len(missing)

1289

In [124]:
missing = missing.to_crs({'init': 'EPSG:7794'})
mask = gpd.GeoDataFrame(geometry = missing.buffer(10))
mask['Dissolve'] = 0
mask = mask.dissolve('Dissolve')
mask = mask.to_crs({'init': 'EPSG:4326'})

In [125]:
clip = sjoin(pts, mask, how='inner', op='within')
print(len(clip))
clip.head()

1289


,geometry,index_right
0,POINT (15.22870738442634 36.32827059812602),0
1,POINT (14.79123570577692 36.38562740834784),0
2,POINT (15.55044433927199 36.4481269376619),0
3,POINT (14.36397979581136 36.46617121034787),0
4,POINT (13.33590466288859 36.39285732614894),0


In [126]:
clip.to_file('../missing_' + n + '.geojson', driver='GeoJSON')
#mask.to_file('../missing_' + n + '_buff.geojson', driver='GeoJSON')